# <img src="https://img.icons8.com/bubbles/50/000000/mind-map.png" style="height:50px;display:inline"> ECE 046211 - Technion - Deep Learning
---

## Project
---

In [ ]:
%pip install datasets
%pip install transformers
%pip install accelerate
%pip install evaluate
%pip install gc

In [1]:
from datasets import load_dataset, Audio,DatasetDict
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer,AutoConfig
import evaluate
import torch
import gc

c:\Users\user\anaconda3\envs\tech046211\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\tech046211\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
gtzan = load_dataset("marsyas/gtzan", "all",trust_remote_code=True)
train_rest = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.2)
val_test = train_rest["test"].train_test_split(seed=42, shuffle=True, test_size=0.5)
gtzan=DatasetDict({'train': train_rest["train"],'val': val_test["train"],'test': val_test["test"]})

In [3]:
id2label_fn = gtzan["train"].features["genre"].int2str

In [4]:
model_id = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
sampling_rate = feature_extractor.sampling_rate
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [5]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [6]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

In [7]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
id2label = {
    i: id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

In [8]:
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [9]:
gc.collect()
torch.cuda.empty_cache()

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(model_id)
model.generation_config=AutoConfig.from_pretrained(model_id)
model.generation_config.id2label=id2label
model.generation_config.label2id=label2id
model.requires_grad_(False)
model.num_labels=num_labels
model.classifier.dense= torch.nn.Linear(model.classifier.dense.in_features,num_labels)

model_name = model_id.split("/")[-1]
batch_size = 64
gradient_accumulation_steps = 1
num_train_epochs = 50

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
    save_total_limit=5,
)
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"].with_format("torch"),
    eval_dataset=gtzan_encoded["val"].with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\user\anaconda3\envs\tech046211\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\user\anaconda3\envs\tech046211\lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|          | 0/650 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\tech046211\lib\site-packages\transformers\models\audio_spectrogram_transformer\modeling_audio_spectrogram_transformer.py:187: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(
  1%|          | 5/650 [00:39<1:01:38,  5.73s/it]

{'loss': 2.613, 'grad_norm': 6.291839122772217, 'learning_rate': 3.846153846153847e-06, 'epoch': 0.38}


  2%|▏         | 10/650 [00:56<39:07,  3.67s/it] 

{'loss': 2.6684, 'grad_norm': 7.667351245880127, 'learning_rate': 7.692307692307694e-06, 'epoch': 0.77}


                                                
  2%|▏         | 13/650 [01:05<23:56,  2.25s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024}


...

                                                 
100%|██████████| 650/650 [28:47<00:00,  1.82s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024}


{'eval_loss': 0.7748748660087585, 'eval_accuracy': 0.8, 'eval_runtime': 3.6867, 'eval_samples_per_second': 27.124, 'eval_steps_per_second': 0.542, 'epoch': 50.0}


100%|██████████| 650/650 [28:48<00:00,  2.66s/it]

{'train_runtime': 1728.7528, 'train_samples_per_second': 23.109, 'train_steps_per_second': 0.376, 'train_loss': 1.177377460186298, 'epoch': 50.0}


TrainOutput(global_step=650, training_loss=1.177377460186298, metrics={'train_runtime': 1728.7528, 'train_samples_per_second': 23.109, 'train_steps_per_second': 0.376, 'total_flos': 2.708117737046016e+18, 'train_loss': 1.177377460186298, 'epoch': 50.0})

In [11]:
pred=trainer.predict(gtzan_encoded["test"].with_format("torch"))
pred.metrics

100%|██████████| 2/2 [00:00<00:00,  2.42it/s]


{'test_loss': 0.839857816696167,
 'test_accuracy': 0.84,
 'test_runtime': 3.4015,
 'test_samples_per_second': 29.399,
 'test_steps_per_second': 0.588}